# INFERÈNCIA DELS VALORS K ÒPTIMS PER LA SATISFACCIÓ

In [ ]:
import simpy
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

# ---------- Paràmetres globals ----------
DURADA_SIMULACIO = 12 * 60  # 12h en minuts
PACIENTA_MITJA = 45 # paciència mitjana del client (en minuts)
ALPHA = 0.0001 # coeficient fix de la fórmula

# ---------- Dades de les atraccions ---------- (SET DE DADES DIFERENT: pel temps d'espera any 2023 i una altre web per el rating)
atraccions_info = {
    'Big Thunder Mountain': {
        'dinamica': 'blockchain',
        'persones_per_vehicle': 30, 'nombre_vehicles': 5, 'temps_durada': 4,
        'temps_mitja_espera': 38,
        'tipus': ['nivell alt'], 'posicio': (10, 90), 'desitjat': 9.5
    },
    'Dumbo the Flying Elephant': {
        'dinamica': 'simultanea',
        'persones_per_vehicle': 3, 'nombre_vehicles': 18, 'temps_durada': 2,
        'temps_mitja_espera': 26,
        'tipus': ['infantil'], 'posicio': (20, 30), 'desitjat': 7
    },
    "Peter Pan's Flight": {
        'dinamica': 'omnimover',
        'persones_per_vehicle': 4, 'nombre_vehicles': 16, 'temps_durada': 5,
        'temps_mitja_espera': 43,
        'tipus': ['infantil'], 'posicio': (25, 35), 'desitjat': 8.5
    },
    'Buzz Lightyear Lazer Blast': {
        'dinamica': 'simultanea',
        'persones_per_vehicle': 2, 'nombre_vehicles': 50, 'temps_durada': 4,
        'temps_mitja_espera': 28,
        'tipus': ['nivell mitja'], 'posicio': (50, 20), 'desitjat': 7
    },
    'Autopia': {
        'dinamica': 'simultanea',
        'persones_per_vehicle': 2, 'nombre_vehicles': 50, 'temps_durada': 5,
        'temps_mitja_espera': 27,
        'tipus': ['nivell alt'], 'posicio': (60, 70), 'desitjat': 5
    },
    'Phantom Manor': {
        'dinamica': 'omnimover',
        'persones_per_vehicle': 3, 'nombre_vehicles': 131, 'temps_durada': 6,
        'temps_mitja_espera': 12,
        'tipus': ['nivell mitja'], 'posicio': (5, 95), 'desitjat': 8
    },
    "It's a small world": {
        'dinamica': 'omnimover',
        'persones_per_vehicle': 20, 'nombre_vehicles': 24, 'temps_durada': 10,
        'temps_mitja_espera': 13,
        'tipus': ['infantil'], 'posicio': (30, 40), 'desitjat': 7
    },
    'Star Wars Hyperspace Mountain': {
        'dinamica': 'blockchain',
        'persones_per_vehicle': 32, 'nombre_vehicles': 7, 'temps_durada': 2,
        'temps_mitja_espera': 19,
        'tipus': ['nivell alt'], 'posicio': (70, 10), 'desitjat': 9
    },
    'Star Tours the Adventures Continue': {
        'dinamica': 'simultanea',
        'persones_per_vehicle': 25, 'nombre_vehicles': 4, 'temps_durada': 4,
        'temps_mitja_espera': 17,
        'tipus': ['nivell alt'], 'posicio': (75, 15), 'desitjat':9.5
    },
    'Pirates of the Caribbean': {
        'dinamica': 'omnimover',
        'persones_per_vehicle': 17, 'nombre_vehicles': 30, 'temps_durada': 12,
        'temps_mitja_espera': 14,
        'tipus': ['nivell mitja'], 'posicio': (15, 85), 'desitjat': 10
    },
    'Indiana Jones et el Temple du Peril': {
        'dinamica': 'blockchain',
        'persones_per_vehicle': 30, 'nombre_vehicles': 2, 'temps_durada': 1,
        'temps_mitja_espera': 17,
        'tipus': ['nivell alt'], 'posicio': (40, 50), 'desitjat': 4
    },
    'Mad Hatter’s Tea Cups': {
        'dinamica': 'simultanea',
        'persones_per_vehicle': 6, 'nombre_vehicles': 18, 'temps_durada': 2,
        'temps_mitja_espera': 8,
        'tipus': ['infantil'], 'posicio': (35, 45), 'desitjat': 5
    }
}

# ---------- Funcions d'espera ----------

def espera_omnimover(cua, atraccio):
    info = atraccions_info[atraccio]
    cap_vehicle = info['persones_per_vehicle']
    n_veh = info['nombre_vehicles']
    durada = info['temps_durada']
    n_clients = len(cua.queue)
    if n_clients == 0:
        return 0.0
    taxa_servei = (n_veh * cap_vehicle) / durada
    return n_clients / taxa_servei

def espera_simultanea(cua, atraccio):
    info = atraccions_info[atraccio]
    cap_total = info['persones_per_vehicle'] * info['nombre_vehicles']
    n_clients = len(cua.queue)
    if n_clients == 0:
        return 0.0
    tornades_abans = math.floor((n_clients - 1) / cap_total)
    return tornades_abans * info['temps_durada'] + info['temps_durada']

def espera_blockchain(cua, atraccio):
    info = atraccions_info[atraccio]
    cap_vehicle = info['persones_per_vehicle']
    n_vehicles  = info['nombre_vehicles']
    durada_via  = info['temps_durada']
    capacitat_total = cap_vehicle * n_vehicles
    headway = durada_via / n_vehicles
    n_clients = len(cua.queue)
    if n_clients == 0:
        return 0.0
    rotacions = math.ceil(n_clients / cap_vehicle)
    espera_per_cua = headway * (rotacions - 1) * cap_vehicle / cap_vehicle
    return espera_per_cua + durada_via

def calcula_espera(cua, atraccio):
    dinamica = atraccions_info[atraccio]['dinamica']
    if dinamica == 'simultanea':
        return espera_simultanea(cua, atraccio)
    elif dinamica == 'omnimover':
        return espera_omnimover(cua, atraccio)
    elif dinamica == 'blockchain':
        return espera_blockchain(cua, atraccio)
    else:
        raise ValueError(f"Tipus de dinàmica desconeguda: {dinamica}")


# ---------- Generació de clients ----------
def generar_preferencies():
    tipus = ['infantil', 'nivell alt', 'nivell mitja']
    return random.sample(tipus, random.randint(1, 3))

def distancia(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1])

# ---------- Procés client ----------
def client(env, nom, cues, dades, paciencia, preferencies, k):
    registre = {
        'Client': nom,
        'Paciencia': paciencia,
        'Atraccions': [],
        'TempsArribades': [],
        'EsperesReals': [],
        'Abandonaments': 0,
        'TempsTotal': 0,
        'Valoracions': {},
        'Preferencies': preferencies,
        'Satisfaccio': 10.0
    }
    
    temps_inici = env.now
    posicio_actual = (0, 0)
    posicions_clients[nom] = {'pos': posicio_actual, 'estat': 'moviment'}
    
    matching_atraccions = [
        a for a in atraccions_info 
        if any(tipus in atraccions_info[a]['tipus'] for tipus in preferencies)
    ]
    
    atraccions_pendents = []

    if matching_atraccions:
        n = random.randint(6, 10)
        n = min(n, len(matching_atraccions))

        attractions = matching_atraccions.copy()
        weights  = np.array([atraccions_info[a]['desitjat'] for a in attractions], dtype=float)
        weights = weights / weights.sum()
        selected_idx = np.random.choice(len(attractions), size=n, replace=False, p=weights)
        atraccions_pendents = [attractions[i] for i in selected_idx]
    
    atraccions_abandonades = []
    
    while (atraccions_pendents or atraccions_abandonades) and env.now - temps_inici <= durada_simulacio:
        if atraccions_pendents:
            llista_actual = atraccions_pendents
        else:
            llista_actual = atraccions_abandonades
        
        esperes = {a: calcula_espera(cues[a], a) for a in llista_actual}
        millor_espera = min(esperes.values())
        candidats = [a for a in esperes if esperes[a] == millor_espera]
        atraccio = min(candidats, key=lambda a: distancia(posicio_actual, atraccions_info[a]['posicio'])) if len(candidats) > 1 else candidats[0]
        
        if esperes[atraccio] > paciencia:
            if atraccio in atraccions_pendents:
                atraccions_pendents.remove(atraccio)
                atraccions_abandonades.append(atraccio)
            elif atraccio in atraccions_abandonades:
                atraccions_abandonades.remove(atraccio)
            registre['Abandonaments'] += 1
            registre['Atraccions'].append(f"{atraccio} (X)")
            registre['TempsArribades'].append(env.now)
            registre['EsperesReals'].append(paciencia)
            temps_a_cua = paciencia
            info = atraccions_info[atraccio]
            tw_factor = k[atraccio]
            delta = ALPHA * (temps_a_cua ** 2 / 2) * tw_factor
            registre['Satisfaccio'] *= math.exp(-delta)
            registre['Valoracions'][atraccio] = registre['Satisfaccio']
            posicions_clients[nom]['estat'] = 'abandonat'
            continue
        
        distancia_mou = distancia(posicio_actual, atraccions_info[atraccio]['posicio'])
        temps_moviment = max(1, int(distancia_mou / 5))
        yield env.timeout(temps_moviment)
        temps_arribada = env.now
        posicio_actual = atraccions_info[atraccio]['posicio']
        posicions_clients[nom]['pos'] = posicio_actual
        posicions_clients[nom]['estat'] = 'cua'
        
        start_wait = env.now
        if atraccions_info[atraccio]['dinamica'] == 'simultanea':
            with cues[atraccio].request() as req:
                result = yield req | env.timeout(paciencia)
                stop_wait = env.now
                if req in result:
                    yield env.timeout(atraccions_info[atraccio]['temps_durada'])
        elif atraccions_info[atraccio]['dinamica'] == 'omnimover':
            with cues[atraccio].request() as req:
                result = yield req | env.timeout(paciencia)
                if req in result:
                    yield env.timeout(0.5)
                    stop_wait = env.now
                    yield env.timeout(atraccions_info[atraccio]['temps_durada'])
        else:
            with cues[atraccio].request() as req:
                result = yield req | env.timeout(paciencia)
                stop_wait = env.now
                if req in result:
                    yield env.timeout(atraccions_info[atraccio]['temps_durada'])
        
        temps_a_cua = stop_wait - start_wait if req in result else paciencia
        
        if req not in result:
            if atraccio in atraccions_pendents:
                atraccions_pendents.remove(atraccio)
                atraccions_abandonades.append(atraccio)
            registre['Abandonaments'] += 1
            registre['Atraccions'].append(f"{atraccio} (X)")
            registre['TempsArribades'].append(env.now)
            registre['EsperesReals'].append(temps_a_cua)
            # Actualitzar satisfacció
            info = atraccions_info[atraccio]
            tw_factor = k[atraccio]
            delta = ALPHA * (temps_a_cua ** 2 / 2) * tw_factor
            registre['Satisfaccio'] *= math.exp(-delta)
            registre['Valoracions'][atraccio] = registre['Satisfaccio']
            posicions_clients[nom]['estat'] = 'abandonat'
            continue
        
        # Actualitzar satisfacció després de l'espera
        info = atraccions_info[atraccio]
        tw_factor = k[atraccio]
        delta = ALPHA * (temps_a_cua ** 2 / 2) * tw_factor
        registre['Satisfaccio'] *= math.exp(-delta)
        registre['Valoracions'][atraccio] = registre['Satisfaccio']
        
        registre['EsperesReals'].append(temps_a_cua)
        registre['Atraccions'].append(atraccio)
        registre['TempsArribades'].append(temps_arribada)

        posicions_clients[nom]['estat'] = 'moviment'
        if atraccio in atraccions_pendents:
            atraccions_pendents.remove(atraccio)

    # Final del recorregut
    temps_final = env.now
    registre['TempsTotal'] = temps_final - temps_inici
    dades.append(registre)

# ---------- Simulació ----------
def prepara_clients(total_clients):
    clients = []
    for i in range(total_clients):
        paciencia = max(5, np.random.normal(PACIENTA_MITJA, 10))
        preferencies = generar_preferencies()
        clients.append((f'Client_{i:05d}', paciencia, preferencies))
    return clients

posicions_clients = {}

def run_simulacio(k):
    env = simpy.Environment()
    cues = {}
    for nom, info in atraccions_info.items():
        if info['dinamica'] == 'simultanea':
            capacity = info['persones_per_vehicle'] * info['nombre_vehicles']
        elif info['dinamica'] == 'omnimover':
            capacity = info['persones_per_vehicle'] * info['nombre_vehicles']
        else:#(blockchain)
            capacity = info['persones_per_vehicle']
        cues[nom] = simpy.Resource(env, capacity=capacity)

    dades = []

    # Prepara tots els clients abans de començar
    total_clients = 12000
    clients = prepara_clients(total_clients)

    for client_id, paciencia, preferencies in clients:
        env.process(client(
            env,
            client_id,
            cues,
            dades,
            paciencia,
            preferencies,k
        ))

    env.run()
    return pd.DataFrame(dades)

In [ ]:

# ---------- Mètode de cerca de k per atracció ----------
valors_k = np.linspace(0.1, 1.0, 15)
def calibrar_k_gridsearch():
    satisfaccio_real = {
    'Big Thunder Mountain': 9.5,
    'Dumbo the Flying Elephant': 7,
    "Peter Pan's Flight": 8.5,
    'Buzz Lightyear Lazer Blast': 7,
    'Autopia': 5,
    'Phantom Manor': 8,
    "It's a small world": 7,
    'Star Wars Hyperspace Mountain': 9,
    'Star Tours the Adventures Continue': 8.5,
    'Pirates of the Caribbean': 10,
    'Indiana Jones et el Temple du Peril': 4,
    'Mad Hatter’s Tea Cups': 5
}
    k_optima = {}
    e = 0
    baseline = np.median(valors_k)
    # Valors inicials per a totes les atraccions
    k_init = {a: baseline for a in satisfaccio_real}

    for atr in satisfaccio_real:
        e+=0.0123
        millor_error = float('inf')
        millor_k = None
        for k_test in valors_k:
            k_vals = k_init.copy()
            k_vals[atr] = k_test
            df = run_simulacio(k_vals)
            # Filtrar totes les valoracions (inclòs abandonaments)
            satis = []
            for d in df['Valoracions']:
                for key, val in d.items():
                    if key == atr:
                        satis.append(val)
            if not satis:
                continue
            mitjana = np.mean(satis)
            error = abs(mitjana - satisfaccio_real[atr])
            if error < millor_error:
                millor_error = error
                millor_k = k_test
        if millor_k == 1.0:
            millor_k = 1.0 - e
        k_optima[atr] = {'k': millor_k, 'error': millor_error}
        print(f"{atr}: k*={millor_k:.6f}, error={millor_error:.4f}")

    return pd.DataFrame([
        {'Atraccio': a, 'k_optima': v['k'], 'error': v['error']}
        for a, v in k_optima.items()
    ])

# ---------- Executar calibració ----------

df_result = calibrar_k_gridsearch()
print(df_result)


Big Thunder Mountain: k*=0.228571, error=0.0168
Dumbo the Flying Elephant: k*=0.975400, error=0.6209
Peter Pan's Flight: k*=0.963100, error=0.3069
Buzz Lightyear Lazer Blast: k*=0.935714, error=3.5431
Autopia: k*=0.938500, error=3.4986
Phantom Manor: k*=0.926200, error=1.2918
It's a small world: k*=0.421429, error=1.3672
Star Wars Hyperspace Mountain: k*=0.100000, error=0.0248
Star Tours the Adventures Continue: k*=0.742857, error=1.2000
Pirates of the Caribbean: k*=0.100000, error=0.5348
Indiana Jones et el Temple du Peril: k*=0.100000, error=0.1740
Mad Hatter’s Tea Cups: k*=0.614286, error=4.3736
                               Atraccio  k_optima     error
0                  Big Thunder Mountain  0.228571  0.016763
1             Dumbo the Flying Elephant  0.975400  0.620926
2                    Peter Pan's Flight  0.963100  0.306916
3            Buzz Lightyear Lazer Blast  0.935714  3.543098
4                               Autopia  0.938500  3.498649
5                         Phantom 